In [ ]:
!pip install pandas

In [89]:
import os 
import pandas as pd
import numpy as np
from datetime import datetime
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hayleyluo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/hayleyluo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hayleyluo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/hayleyluo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [90]:
# Get the current directory
current_directory = os.getcwd()

print(f"current directory: {current_directory}")

current directory: /Users/hayleyluo/Desktop/Github/ML-kaggle-projects/kaggle-ski-area-review


In [91]:
file = f"{current_directory}/datasets/raw/OnTheSnow_SkiAreaReviews.csv"
df = pd.read_csv(file, header=0)
df

,Unnamed: 0,State,Ski Area,Reviewer Name,Review Date,Review Star Rating (out of 5),Review Text
0,0,california,squaw-valley-usa,philip sayles,31st December 2019,3,I'm glad our family experienced Squaw but I wo...
1,1,california,squaw-valley-usa,mateonelson,22nd May 2019,4,"I went skiing today 5/22, granite chief was am..."
2,2,california,squaw-valley-usa,NaN,7th March 2019,2,We had a horrible experience on our family ski...
3,3,california,squaw-valley-usa,Ivan Cazares,5th March 2019,3,This is the first year I ski Squaw. I've been ...
4,4,california,squaw-valley-usa,welzbob,26th February 2019,3,"Both Squaw and Alpine have incredible terrain,..."
...,...,...,...,...,...,...,...
18257,18257,new-hampshire,whaleback-mountain,Matthijs,18th March 2017,1,Looks like they only made snow on one trail. Y...
18258,18258,new-hampshire,whaleback-mountain,Dolly,25th February 2017,3,Looks cute from highway........ but not enough...
18259,18259,new-hampshire,whaleback-mountain,Roger,11th February 2017,3,Place has incredible potential. Could be best ...
18260,18260,new-hampshire,whaleback-mountain,Arthur,10th February 2017,2,Great exposure from interstate......... but lo...


In [92]:
# Set the first column ('col1') as the index
df = df.drop(df.columns[0], axis=1)  # Drop by index
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18262 entries, 0 to 18261
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   State                          18262 non-null  object
 1   Ski Area                       18262 non-null  object
 2   Reviewer Name                  18153 non-null  object
 3   Review Date                    18262 non-null  object
 4   Review Star Rating (out of 5)  18262 non-null  int64 
 5   Review Text                    18250 non-null  object
dtypes: int64(1), object(5)
memory usage: 856.2+ KB


In [93]:
# impute the missing value in Reviewer Name with anonymous_user
df['Reviewer Name'].fillna("anonymous_user", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18262 entries, 0 to 18261
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   State                          18262 non-null  object
 1   Ski Area                       18262 non-null  object
 2   Reviewer Name                  18262 non-null  object
 3   Review Date                    18262 non-null  object
 4   Review Star Rating (out of 5)  18262 non-null  int64 
 5   Review Text                    18250 non-null  object
dtypes: int64(1), object(5)
memory usage: 856.2+ KB


In [94]:
# drop the rows that has no review text. 
df = df.dropna(subset=["Review Text"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18250 entries, 0 to 18261
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   State                          18250 non-null  object
 1   Ski Area                       18250 non-null  object
 2   Reviewer Name                  18250 non-null  object
 3   Review Date                    18250 non-null  object
 4   Review Star Rating (out of 5)  18250 non-null  int64 
 5   Review Text                    18250 non-null  object
dtypes: int64(1), object(5)
memory usage: 998.0+ KB


In [95]:
# impute the missing rating with the rating mean of the dataset
impute_mean = df["Review Star Rating (out of 5)"].mean()
df["Review Star Rating (out of 5)"] = df["Review Star Rating (out of 5)"].fillna(impute_mean).values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18250 entries, 0 to 18261
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   State                          18250 non-null  object
 1   Ski Area                       18250 non-null  object
 2   Reviewer Name                  18250 non-null  object
 3   Review Date                    18250 non-null  object
 4   Review Star Rating (out of 5)  18250 non-null  int64 
 5   Review Text                    18250 non-null  object
dtypes: int64(1), object(5)
memory usage: 998.0+ KB


In [96]:
def convert_date(date_str):
    """
    convert string to data to python data time object
    arg:
        date_str: data format in 1st October 2026
    return:
        converted python data time object
    """
    # pattern match the data format e.g. 1st October 2026
    pattern = r"\b(\d{1,2})(st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b"
    match = re.search(pattern, date_str)
    date_list = match.group().split(" ")
    # remove 'st','nd','ed','th' from day
    output_str = f"{date_list[0].replace('st', '').replace('nd', '').replace('rd', '').replace('th', '')} {date_list[1]} {date_list[2]}"
    return pd.to_datetime(output_str, format='%d %B %Y')

# convert the data time string column
df['Review Date datetime'] = df['Review Date'].apply(convert_date)
df = df.drop(columns=["Review Date"])

In [97]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # make lower cases
    text = str(text).lower()
    # Retain only alphanumeric characters and spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', "", text)
    # strip extra white space
    text = re.sub(r"\s+", " ", text).strip()
    # tokenize works into a list
    tokens = word_tokenize(text)
    # remove the stop words and lemmatize the remaining words
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    # return the clean text
    return " ".join(tokens)

df["Review Text Cleaned"] = df["Review Text"].apply(clean_text)

df = df.drop(columns=["Review Text"])
df = df.dropna(subset=["Review Text Cleaned"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18250 entries, 0 to 18261
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   State                          18250 non-null  object        
 1   Ski Area                       18250 non-null  object        
 2   Reviewer Name                  18250 non-null  object        
 3   Review Star Rating (out of 5)  18250 non-null  int64         
 4   Review Date datetime           18250 non-null  datetime64[ns]
 5   Review Text Cleaned            18250 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 998.0+ KB


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18250 entries, 0 to 18261
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   State                          18250 non-null  object        
 1   Ski Area                       18250 non-null  object        
 2   Reviewer Name                  18250 non-null  object        
 3   Review Star Rating (out of 5)  18250 non-null  int64         
 4   Review Date datetime           18250 non-null  datetime64[ns]
 5   Review Text Cleaned            18250 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 998.0+ KB


In [99]:
file_path = f'{current_directory}/datasets/bronze/on_the_snow_ski_area_reviews.csv'
df.to_csv(file_path, header=True, index=True, sep=',', encoding='utf-8')
print(f"save dataset to {file_path}")

save dataset to /Users/hayleyluo/Desktop/Github/ML-kaggle-projects/kaggle-ski-area-review/datasets/bronze/on_the_snow_ski_area_reviews.csv


In [100]:
df_read = pd.read_csv(file_path, header=0, index_col=0, sep=',', encoding='utf-8')
print(f"read dataset from {file_path}")    

df_read.info()

read dataset from /Users/hayleyluo/Desktop/Github/ML-kaggle-projects/kaggle-ski-area-review/datasets/bronze/on_the_snow_ski_area_reviews.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18250 entries, 0 to 18261
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   State                          18250 non-null  object
 1   Ski Area                       18250 non-null  object
 2   Reviewer Name                  18250 non-null  object
 3   Review Star Rating (out of 5)  18250 non-null  int64 
 4   Review Date datetime           18250 non-null  object
 5   Review Text Cleaned            18199 non-null  object
dtypes: int64(1), object(5)
memory usage: 998.0+ KB
